# Prerequisites

## Create Delta Table

In [0]:
%sql
DROP TABLE quickstart_catalog.quickstart_schema.users_int;

In [0]:
%sql

CREATE TABLE quickstart_catalog.quickstart_schema.users_int(
  id INT,
  name STRING,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
) USING DELTA;

DESCRIBE EXTENDED quickstart_catalog.quickstart_schema.users_int

# Data Ingestion using Autoloader

## Scenario 01

In [0]:
spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_data/sandbox/dataset/users_staging_cloudfiles/")

### Using option function (key: value, key: value)

In [0]:
df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", "/FileStore/tables/_sqldf")
    .option("cloudFiles.schemaEvolutionMode", "resolveSchema")
    .option("cloudFiles.maxFilesPerTrigger", 1)
    .load("/FileStore/tables/_sqldf")
)
display(df)

### Using options function

In [0]:
cloud_files_properties = {
    "cloudFiles.format": "csv",
    "cloudFiles.schemaLocation": "/Volumes/quickstart_catalog/quickstart_schema/sandbox/schema/",
    "header": "true",
    "cloudFiles.inferColumnTypes": "true",
    "timestampFormat": "yyyy-MM-dd",
}

df = (
    spark.readStream.format("cloudFiles")
    .options(**cloud_files_properties)
    .option(
        "checkpointLocation",
        "/Volumes/quickstart_catalog/quickstart_checkpoint/sandbox/cloudfiles_checkpoint",
    )
    .load(
        "/Volumes/quickstart_catalog/quickstart_data/sandbox/dataset/users_staging_cloudfiles/"
    )
)

In [0]:
query = (
    df.writeStream
        .format("delta")
        .outputMode("append")  
        .option("checkpointLocation", "/Volumes/quickstart_catalog/quickstart_schema/sandbox/cloudfiles_checkpoint")
        .option("mergeSchema", "true")
        .trigger(availableNow=True)  
        .toTable("quickstart_catalog.quickstart_schema.users_int")
)

In [0]:
spark.table("quickstart_catalog.quickstart_schema.users_int").display()